In [7]:
from google.colab import auth
from google.auth.transport.requests import Request
import gspread
import pandas as pd
import re
from datetime import datetime
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, Border, Side

# 1. Đăng nhập vào tài khoản Google của bạn
auth.authenticate_user()

# 2. Sử dụng thông tin xác thực mặc định của Colab
creds, _ = default()

# 3. Làm mới thông tin xác thực nếu cần
creds.refresh(Request())

# 4. Kết nối với Google Sheets
gc = gspread.authorize(creds)

# 5. Mở Google Sheet bằng URL
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1rdcNBfwBGkeMxrpGWo77I8Eyi1_3H9-57XQNOsJ-_cM/edit?gid=1968705733')

# 6. Chọn trang tính (worksheet) đầu tiên
worksheet = sheet.get_worksheet(0)

# 7. Lấy tất cả dữ liệu từ trang tính
all_data = worksheet.get_all_values()

# 8. Chuyển dữ liệu thành DataFrame
df = pd.DataFrame(all_data[1:], columns=all_data[0])

# 9. Loại bỏ khoảng trắng dư thừa trong tên cột
df.columns = df.columns.str.strip()

# 10. Chuyển đổi số liệu trong các cột cụ thể
def convert_to_int(value):
    value = re.sub(r'\D', '', value)  # Loại bỏ ký tự không phải số
    return int(value) if value.isdigit() else 0

df['Số tiết thực giảng'] = df['Số tiết thực giảng'].apply(convert_to_int)
df['Số lượng HSSV có mặt'] = df['Số lượng HSSV có mặt'].apply(convert_to_int)
df['Số lượng HSSV Vắng - Có lý do'] = df['Số lượng HSSV Vắng - Có lý do'].apply(convert_to_int)
df['Số lượng HSSV Vắng - Không có lý do'] = df['Số lượng HSSV Vắng - Không có lý do'].apply(convert_to_int)

# 11. Lọc dữ liệu theo cột "Dấu thời gian"
today = datetime.today().strftime('%d/%m/%Y')
df_today = df[df['Dấu thời gian'].str.contains(today, na=False)].copy()

# 12. Thêm cột "Buổi"
def determine_session(time_str):
    try:
        hour_str = time_str.split()[1].split(':')[0]
        hour = int(hour_str.zfill(2))  # Đảm bảo giờ có 2 chữ số (ví dụ: '7' thành '07')
        if 7 <= hour <= 11:
            return 'Sáng'
        else:
            return 'Chiều'
    except (ValueError, IndexError):
        return 'Không xác định'  # Xử lý lỗi khi dữ liệu không đúng định dạng

df_today.loc[:, 'Buổi'] = df_today['Dấu thời gian'].apply(determine_session)

# 13. Tạo DataFrame riêng cho buổi sáng và buổi chiều
df_morning = df_today[df_today['Buổi'] == 'Sáng'].copy()
df_afternoon = df_today[df_today['Buổi'] == 'Chiều'].copy()

# 14. Thêm cột số thứ tự (STT)
df_morning.insert(0, 'STT', range(1, len(df_morning) + 1))
df_afternoon.insert(0, 'STT', range(1, len(df_afternoon) + 1))

# 15. Gộp lại hai DataFrame buổi sáng và chiều
daily_report = pd.concat([df_morning, df_afternoon])

# 16. Sắp xếp các cột theo yêu cầu và thêm cột "Cán bộ chấm" đầu tiên và "Ghi chú" cuối cùng
report_columns = [
    'STT',
    'Cán bộ chấm',
    'Họ tên giáo viên',
    'Môn giảng theo TKB',
    'Lớp',
    'Buổi',
    'Số tiết thực giảng',
    'Số lượng HSSV có mặt',
    'Số lượng HSSV Vắng - Có lý do',
    'Số lượng HSSV Vắng - Không có lý do',
    'Ghi chú'
]
daily_report = daily_report[report_columns]

# 17. Tạo workbook mới và sheet
wb = Workbook()
ws = wb.active

# 18. Thêm tiêu đề báo cáo (in hoa)
ws.merge_cells('A1:K1')
ws['A1'] = 'BÁO CÁO HÀNG NGÀY'.upper()
ws['A1'].font = Font(size=14, bold=True)
ws['A1'].alignment = Alignment(horizontal='center')

# 19. Thêm ngày tháng
ws.merge_cells('A2:K2')
ws['A2'] = f'Ngày: {today}'
ws['A2'].font = Font(size=14, bold=True)
ws['A2'].alignment = Alignment(horizontal='center')

# 20. Thêm dữ liệu từ DataFrame vào Excel và định dạng bảng
thin_border = Border(left=Side(style='thin'),
                     right=Side(style='thin'),
                     top=Side(style='thin'),
                     bottom=Side(style='thin'))

for r_idx, row in enumerate(dataframe_to_rows(daily_report, index=False, header=True), 4):
    for c_idx, value in enumerate(row, 1):
        cell = ws.cell(row=r_idx, column=c_idx, value=value)
        cell.border = thin_border
        cell.alignment = Alignment(horizontal='center')
        if r_idx == 4:  # Định dạng tiêu đề cột
            cell.font = Font(bold=True)

# 21. Thêm dòng tổng cộng cho từng buổi (Sáng và Chiều)
def add_total_row(session):
    session_data = daily_report[daily_report['Buổi'] == session]
    if not session_data.empty:
        total_row = ws.max_row + 1
        ws.cell(row=total_row, column=6, value=f"Tổng cộng {session}").font = Font(bold=True)
        ws.cell(row=total_row, column=6).alignment = Alignment(horizontal='center')
        ws.cell(row=total_row, column=7, value=session_data['Số tiết thực giảng'].sum()).font = Font(bold=True)
        ws.cell(row=total_row, column=7).alignment = Alignment(horizontal='center')
        ws.cell(row=total_row, column=8, value=session_data['Số lượng HSSV có mặt'].sum()).font = Font(bold=True)
        ws.cell(row=total_row, column=8).alignment = Alignment(horizontal='center')
        ws.cell(row=total_row, column=9, value=session_data['Số lượng HSSV Vắng - Có lý do'].sum()).font = Font(bold=True)
        ws.cell(row=total_row, column=9).alignment = Alignment(horizontal='center')
        ws.cell(row=total_row, column=10, value=session_data['Số lượng HSSV Vắng - Không có lý do'].sum()).font = Font(bold=True)
        ws.cell(row=total_row, column=10).alignment = Alignment(horizontal='center')

# Thêm dòng tổng cộng cho buổi sáng và chiều
add_total_row('Sáng')
add_total_row('Chiều')

# 22. Định dạng cột và dòng
for col in ws.iter_cols(min_row=4, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    max_length = 0
    column = col[0].column_letter  # Lấy chữ cái của cột
    for cell in col:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))
    adjusted_width = max_length + 2
    ws.column_dimensions[column].width = adjusted_width

# 23. Thêm ghi chú cuối bảng
last_row = ws.max_row + 2
ws.merge_cells(f'A{last_row}:K{last_row}')
ws[f'A{last_row}'] = "PHÒNG THANH TRA - CÔNG TÁC HSSV".upper()
ws[f'A{last_row}'].font = Font(bold=True)
ws[f'A{last_row}'].alignment = Alignment(horizontal='right')

# 24. Lưu file Excel vào thư mục hiện tại với ngày trong tên file
file_name = f'Báo cáo ngày_{today.replace("/", "-")}.xlsx'
wb.save(file_name)

# Lưu dữ liệu thành file CSV
csv_file_name = f'Báo cáo ngày_{today.replace("/", "-")}.csv'
daily_report.to_csv(csv_file_name, index=False)

# Thông báo về kết quả lưu file
print(f"Báo cáo hàng ngày đã được lưu thành file Excel: {file_name}")
print(f"Báo cáo hàng ngày đã được lưu thành file CSV: {csv_file_name}")



Báo cáo hàng ngày đã được lưu thành file Excel: Báo cáo ngày_12-08-2024.xlsx
Báo cáo hàng ngày đã được lưu thành file CSV: Báo cáo ngày_12-08-2024.csv
